In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
SCOPETYPE = 'OPENvADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

scope.reload_fpga("../../cwlite_interface_ec.bit")

%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np

from holoviews.streams import Pipe, Buffer
from scipy.ndimage.filters import uniform_filter1d
from tqdm.notebook import trange

import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
from chipwhisperer.common.traces import Trace

class real_time_plot:
    def __init__(self, plot_len, title="Plot"):
        hv.extension('bokeh')
        st = pd.DataFrame({'y':[]}, columns=['y'])
        self.dfstream = Buffer(st, length=plot_len, index=False)
        curve_dmap = hv.DynamicMap(hv.Curve, streams=[self.dfstream])
        curve_dmap.opts(tools=["hover"], width=800, height=600, title=title)
        display(curve_dmap)
        self.curve = curve_dmap
    def send(self, data):
        if hasattr(data, 'wave'):
            d = data.wave
        else:
            d = data
        self.dfstream.send(pd.DataFrame({'y':d}))

def powercycle_target(scope):
    scope.io.tio1 = "gpio_low"
    time.sleep(0.0001)
    scope.io.tio1 = "gpio_high"

def setup_sad():
    scope.SAD.reference = np.ones(128)*0.4
#     print(f"cw reports SAD.reference: {scope.SAD.reference}")

    print(scope.SAD.check_status())
    scope.SAD.threshold = 5000
    print(f"cw reports SAD.threshold: {scope.SAD.threshold}")

def print_ec_conf():
    print("==== Trigger ====")
    print(f"cw reports running:\t\t {scope.EC.check_status()}")
    print(f"cw reports edge_num:\t\t {scope.EC.edge_num}")
    print(f"cw reports settling_time:\t {scope.EC.settling_time}")
    print(f"cw reports pretrigger_ctr:\t {scope.EC.pretrigger_ctr}")
    print(f"cw reports edge_type:\t\t {scope.EC.edge_type}")
    print(f"cw reports threshold:\t\t {scope.EC.threshold}")
    print(f"cw reports trigger module:\t {scope.trigger.module}")
    
def setup_ec():
    scope.EC.threshold = 0.016
    scope.EC.edge_type = "falling_edge"
    scope.EC.settling_time = 127
    scope.EC.pretrigger_ctr = 1
    scope.EC.edge_num = 1
    
    scope.clock.adc_src = "clkgen_x1"

    scope.adc.basic_mode = "low"
    scope.trigger.module = "basic"
    scope.trigger.triggers = "tio4"

    scope.adc.offset = offset
    scope.gain.gain = 50
    scope.adc.samples = nr_samples
    scope.adc.presamples = 2000
    
def run_test_cycle():
    plot = real_time_plot(plot_len=20000)

    for i in trange(1, desc='Capturing traces'):    
        powercycle_target(scope)
        scope.arm()
        scope.capture()
        trace = scope.get_last_trace()
        if trace is None:
            continue
        plot.send(trace)

nr_samples = 24400  
offset = 0
setup_ec()
print_ec_conf()
# setup_sad()
scope.trigger.triggers = "tio4"
run_test_cycle()